In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds


In [ ]:
!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

--2020-06-28 07:07:50--  https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  22.4MB/s    in 5.5s    

2020-06-28 07:07:56 (14.6 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [ ]:
!tar -xf aclImdb_v1.tar.gz

In [ ]:
!ls

aclImdb  aclImdb_v1.tar.gz  sample_data


In [ ]:
dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True, as_supervised=True)

train_dataset, test_dataset = dataset['train'], dataset['test']

encoder = info.features['text'].encoder

In [ ]:
type(encoder)

tensorflow_datasets.core.features.text.subword_text_encoder.SubwordTextEncoder

In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [ ]:
padded_shapes = ([None], ())

In [ ]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE, 
                                                                padded_shapes=padded_shapes)

In [ ]:
test_dataset = test_dataset.padded_batch(BATCH_SIZE, padded_shapes=padded_shapes)

In [ ]:
model = tf.keras.Sequential([tf.keras.layers.Embedding(encoder.vocab_size, 64),
                             tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
                             tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
                             tf.keras.layers.Dense(64, activation= 'relu'),
                             tf.keras.layers.Dropout(0.5),
                             tf.keras.layers.Dense(1,activation="sigmoid")])

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

history = model.fit(train_dataset, epochs=10, validation_data = test_dataset,
                    validation_steps=30)

def pad_to_size(vec, size):
  zeros = [0]*(size-len(vec))
  vec.extend(zeros)
  return vec

def sample_predict(sentence, pad):
  encoded_sample_pred_text = encoder.encode(sentence)
  if pad:
    encoded_sample_pred_text = pad_to_size(encoded_sample_pred_text, 64)
  encoded_sample_pred_text = tf.cast(encoded_sample_pred_text, tf.float32)
  predictions = model.predict(tf.expand_dims(encoded_sample_pred_text, 0))
  return predictions


sample_text = ('This movie was emotional. The actors were perfect. A must watch')
predictions = sample_predict(sample_text, pad=True)*10

Epoch 1/10
391/391 [==============================] - 78s 201ms/step - loss: 0.6761 - accuracy: 0.5692 - val_loss: 0.5420 - val_accuracy: 0.7401
Epoch 2/10
391/391 [==============================] - 80s 205ms/step - loss: 0.3990 - accuracy: 0.8385 - val_loss: 0.3649 - val_accuracy: 0.8516
Epoch 3/10
391/391 [==============================] - 81s 207ms/step - loss: 0.2812 - accuracy: 0.9007 - val_loss: 0.3452 - val_accuracy: 0.8719
Epoch 4/10
391/391 [==============================] - 82s 208ms/step - loss: 0.2262 - accuracy: 0.9250 - val_loss: 0.3443 - val_accuracy: 0.8766
Epoch 5/10
391/391 [==============================] - 81s 207ms/step - loss: 0.1950 - accuracy: 0.9392 - val_loss: 0.3710 - val_accuracy: 0.8661
Epoch 6/10
391/391 [==============================] - 80s 204ms/step - loss: 0.1678 - accuracy: 0.9508 - val_loss: 0.3799 - val_accuracy: 0.8672
Epoch 7/10
391/391 [==============================] - 80s 205ms/step - loss: 0.1434 - accuracy: 0.9603 - val_loss: 0.4257 - val_ac

In [ ]:
print('Predicted review based on IMDB data%.2f' %predictions)

Predicted review based on IMDB data9.84


In [ ]:
#transfer to django web app for real time consumer feedback